# Calculateur d'écriture pour Atoutprises

Lancer l'exécution des cellules en cliquant sur `Tout exécuter` (Run all) depuis le menu `Exécution` (Runtime). Puis uploader le fichier de dépenses simplifiées dans l'avant dernière cellule.

### Setup

In [ ]:
from IPython.display import clear_output
print('Installation des dépendences...')
!apt install ghostscript python3-tk
!pip install camelot-py[cv] ghostscript tabulate 'PyPDF2<3.0'
clear_output()
print('Installation terminée...')

### Code

Les fonctions suivantes permettent de lire les informations contenues dans le fichier `Etat simplifié des dépenses salariales`

In [ ]:
# @title
import re
import camelot
import pandas as pd
from tabulate import tabulate
from PyPDF2 import PdfReader

In [ ]:
# @title

def read_month(filename: str) -> str:
    """
    Fonction pour extraire le mois à partir d'un fichier PDF.

    Parameters
    ----------
    filename : str
        le nom du fichier

    Returns
    -------
    str
        le mois
    """
    reader = PdfReader(filename)
    text = reader.pages[0].extract_text()
    res = re.search(
        r"Etat simplifié des dépenses salariales : ([a-zA-Z]+) 202[0-9]", text
    )
    if res:
        return res.group(1)
    return ""

In [ ]:
#@title

def read_table(filename: str) -> pd.DataFrame:
    """
    Fonction pour lire les tableaux dans un fichier PDF.

    Parameters
    ----------
    filename : str
        le nom du fichier

    Returns
    -------
    pd.DataFrame
        le tableau contenant les informations de comptabilité
    """
    tables = camelot.read_pdf(filename)
    df = tables[0].df
    if len(tables) == 2:
        df1 = tables[1].df
        df1.index = (0, 2, 3, 4)
        df1.loc[1] = [None] * df1.shape[1]
        df1 = df1.sort_index()
        df = pd.concat((df1, df), axis=1)
    else:
        df = df.drop(0).reset_index(drop=True)

    assert df.iat[2, 0].startswith(
        "CEINTRE Arnaud"
    ), "First row should be CEINTRE Arnaud"
    assert df.shape == (4, 22), "Table should be 4x22"

    return df

In [ ]:
#@title

def read_row(row: int, month: str, df: pd.DataFrame):
    """
    Fonction pour lire les données d'une ligne dans le tableau et
    afficher les lignes pour l'écriture en comptabilité

    Parameters
    ----------
    row : int
        l'index de la ligne à lire dans le tableau
    month : str
        le mois en cours
    df : pd.DataFrame
        le tableau
    """

    # Input values
    name = df.iat[row, 0].split("\n")[0]
    brut = float(df.iat[row, 2])
    net = float(df.iat[row, 18])
    pas = float(df.iat[row, 17])
    urssafPO = float(df.iat[row, 6])
    urssafPP = float(df.iat[row, 7])
    chomagePP = float(df.iat[row, 9])
    retraitePO = float(df.iat[row, 10])
    retraitePP = float(df.iat[row, 11])
    m = re.search(r"L : (\d+(\.\d+)?)", df.iat[row, 12])
    formationL = float(m.group(1))
    prevoyancePO = float(df.iat[row, 13])
    prevoyancePP = float(df.iat[row, 14])

    # Debit values
    charges = urssafPP + chomagePP + retraitePP + formationL + prevoyancePP

    # Credit values
    cotisations = urssafPO + urssafPP + chomagePP + formationL
    retraites = retraitePO + retraitePP
    prevoyances = prevoyancePO + prevoyancePP

    # Print table
    headers = ("Libellé", "Compte", "Débit", "Crédit")
    values = []
    values.append(
        [
            f"Salaire brut {month} {name}",
            "641000 - Rémunérations du personnel",
            "{:.2f}".format(brut),
            "",
        ]
    )
    values.append(
        [
            f"Charges patronales {month} {name}",
            "645100 - Cotisations à l'URSSAF",
            "{:.2f}".format(charges),
            "",
        ]
    )
    values.append(
        [
            f"Salaire net {month} {name}",
            "421000 - Personnel - Rémunérations dues",
            "",
            "{:.2f}".format(net),
        ]
    )
    values.append(
        [
            f"Total cotisations URSSAF {month} {name}",
            "431000 - Sécurité sociale",
            "",
            "{:.2f}".format(cotisations),
        ]
    )
    values.append(
        [
            f"Charges ARRCO {month} {name}",
            "437300 - Caisses de retraites et de prévoyance",
            "",
            "{:.2f}".format(retraites),
        ]
    )
    values.append(
        [
            f"PAS {month} {name}",
            "442100 - Prélèvements à la source (IR)",
            "",
            "{:.2f}".format(pas),
        ]
    )
    values.append(
        [
            f"Prévoyance {month} {name}",
            "437200 - Mutuelles",
            "",
            "{:.2f}".format(prevoyances),
        ]
    )

    values.append(
        [
            "===== total",
            "",
            "{:.2f}".format(brut + charges),
            "{:.2f}".format(net + cotisations + retraites + pas + prevoyances),
        ]
    )
    print(tabulate(values, headers=headers))


### Upload du fichier

Sélectionner le fichier `Etat simplifié des dépenses salariales` pour le mois à l'aide du bouton dans la case suivante.

In [ ]:
#@title

from google.colab import files
uploaded = files.upload()

### Calcul des lignes de comptabilité

In [ ]:
for filename in uploaded.keys():
  df = read_table(filename)
  month = read_month(filename)
  for row in (2,):
    read_row(row, month, df)
    print()
    print()